此ipynb旨在通过两个高德api获取到广州市内所有交叉路口的经纬度数据

我高德的key值为91e0259f802bf676777e5a4d358fb9ed

In [1]:
key = '91e0259f802bf676777e5a4d358fb9ed'

第一个api是查询行政区域的api，官方文档https://lbs.amap.com/api/webservice/guide/api/district

URL：https://restapi.amap.com/v3/config/district?parameters

请求方式：GET

In [2]:
import requests

In [3]:
url = 'https://restapi.amap.com/v3/config/district'
data = {
    'keywords': '广州',
    'subdistrict': 3,
    'key': key
}

In [5]:
r = requests.get(url, params=data)

In [6]:
districts = r.json()['districts'][0]['districts']  # 记录着各区信息，是个列表

In [7]:
streets = districts[0]['districts']  # 记录着街道信息，是个列表

因为访问下一个接口时候的点坐标仍为"lnt, lat"的形式，所以先不进行格式的转换

In [10]:
points = []
for district in districts:
    points.append(district['center'])
    streets = district['districts']
    for street in streets:
#         point = list(map(float, street['center'].split(',')))
        point = street['center']
        points.append(point)

In [11]:
print(len(points))

190


这样我们就得到了广州每个街道及每个区的中心点

此时，第二个api隆重登场，逆地理编码的api，官方文档https://lbs.amap.com/api/webservice/guide/api/georegeo

URL：https://restapi.amap.com/v3/geocode/regeo?parameters

请求方式：GET

In [15]:
second_url = 'https://restapi.amap.com/v3/geocode/regeo'
second_data = {
    'key': key,
    'location': points[0],
    'radius': 3000,
    'extensions': 'all',
}
r2 = requests.get(second_url, params=second_data)
for i in r2.json()['regeocode']['roads']:
    print(i['location'])

113.586,23.5481
113.585,23.5485
113.586,23.548


In [16]:
intersections = []
for point in points:
    second_url = 'https://restapi.amap.com/v3/geocode/regeo'
    second_data = {
        'key': key,
        'location': point,
        'radius': 3000,
        'extensions': 'all',
    }
    r2 = requests.get(second_url, params=second_data)
    # 添加十字路口的经纬度
    for one in r2.json()['regeocode']['roadinters']:
        intersections.append(one['location'])
    # 添加路口的经纬度
    for i in r2.json()['regeocode']['roads']:
        intersections.append(i['location'])

In [20]:
# 查看十字路口数
print(len(intersections))
print(len(list(set(intersections))))

751
702


In [21]:
# 说明确实有些重复的点，还是要进行筛选的
intersections = list(set(intersections))

显然，每个点为字符串，以,分隔，为后面的数据预处理，要将每个点处理成列表形式，方便np.array转二维

In [22]:
list(map(float, intersections[0].split(',')))

[113.249041, 23.147824]

In [23]:
new_intersections = []
for i in range(len(intersections)):
    new_intersections.append(list(map(float, intersections[i].split(','))))

In [24]:
import json
with open('new_intersections.json', 'w') as f:
    a = json.dumps(new_intersections)
    f.write(a)